In [106]:
import cirq
from math import pi

In [107]:
# Assigning qubits that will be required by the circuit
MAX_QUBITS = 5 # We need to know this number before hand to compile an equivalent circuit
N_QUBITS = MAX_QUBITS
circuit = cirq.Circuit()
qubits = [cirq.LineQubit(i) for i in range(N_QUBITS)]

In [108]:
# Creating a sample circuit

# Applying Hadamard Gate
circuit.append([cirq.H(qubits[0])])
circuit.append([cirq.H(qubits[1])])
circuit.append([cirq.H(qubits[2])])

# Applying CNOT Gate
for i in range(n_qubits - 1):
    circuit.append([cirq.CNOT(qubits[i], qubits[i + 1])], cirq.InsertStrategy.NEW)

# Swapping 0 and 4
SWAP = cirq.SWAP(qubits[0], qubits[4])
circuit.append([SWAP])

# Shifting phase along x axis by pi/2 radians
phase_shift = cirq.rx(pi)
ROTATE = cirq.ParallelGateOperation(phase_shift, qubits)
circuit.append([ROTATE])

# Adding a measurement gate
# for q in qubits:
#     circuit.append([cirq.measure(q)])

In [109]:
# Printing the circuit
print(circuit)

0: ───H───@───────────────×───Rx(π)───
          │               │
1: ───H───X───@───────────┼───Rx(π)───
              │           │
2: ───H───────X───@───────┼───Rx(π)───
                  │       │
3: ───────────────X───@───┼───Rx(π)───
                      │   │
4: ───────────────────X───×───Rx(π)───


In [170]:
def compile_moment(moment):
    compiled_operations = []
    for op in moment.operations:
        # print(op)

        gate = str(op).split("(")[0]
        on_qubits = [int(s) for s in str(op) if s.isdigit()]
        if gate == 'I':
            new_op = compile_I(on_qubits)
        elif gate == 'H':
            new_op = compile_H(on_qubits)
        elif gate == 'X':
            new_op = compile_X(on_qubits)
        elif gate == 'Y':
            new_op = compile_Y(on_qubits)
        elif gate == 'Z':
            new_op = compile_Z(on_qubits)
        elif gate == 'Rx':
            new_op = compile_rx(on_qubits)
        elif gate == 'Ry':
            new_op = compile_ry(on_qubits)
        elif gate == 'Rz':
            new_op = compile_rz(on_qubits)
        elif gate == 'CNOT':
            new_op = compile_CNOT(on_qubits)
        elif gate == 'CZ':
            new_op = compile_CZ(on_qubits)
        
        compiled_operations.append(new_op)
    return compiled_operations

In [171]:
new_circuit = cirq.Circuit()
for moment in circuit:
    new_moment = compile_moment(moment)
    new_circuit.append(new_moment)
# print(new_circuit)

H
H
H
CNOT
CNOT
CNOT
CNOT
